# Load and resize images

In [1]:
# train_labels = pd.read_csv('../data/train-lables.csv')

In [2]:
import pandas as pd

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
validation_labels = pd.read_csv('../data/validation-lables.csv')

In [4]:
from skimage.io import imread
from skimage.transform import resize
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_images(input_dir, size):
    filenames = os.listdir(input_dir)
    
    result = []
    
    for filename in filenames:
        image_name = filename.split('.')[0]
        image_id = image_name.split('_')[1]
        try:
            image = imread(os.path.join(input_dir, filename))
            if len(image.shape) != 3 and image.shape[2] == 3:
                raise 'Bad image dimensions'
            result.append([image_id, resize(image, size)])
        except Exception as e:
            print(e)
            print(image_name.split('_')[1])
            
    return result

In [5]:
val_images = load_images('../data/validation/', (299, 299))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


cannot identify image file <_io.BufferedReader name='../data/validation/pic_3015.png'>
3015
cannot identify image file <_io.BufferedReader name='../data/validation/pic_6102.png'>
6102
cannot identify image file <_io.BufferedReader name='../data/validation/pic_4456.png'>
4456
cannot identify image file <_io.BufferedReader name='../data/validation/pic_3023.png'>
3023
cannot identify image file <_io.BufferedReader name='../data/validation/pic_5992.png'>
5992
cannot identify image file <_io.BufferedReader name='../data/validation/pic_5675.png'>
5675
tuple index out of range
5387
cannot identify image file <_io.BufferedReader name='../data/validation/pic_2902.png'>
2902
tuple index out of range
178
cannot identify image file <_io.BufferedReader name='../data/validation/pic_4753.png'>
4753
tuple index out of range
4774
cannot identify image file <_io.BufferedReader name='../data/validation/pic_3733.png'>
3733
cannot identify image file <_io.BufferedReader name='../data/validation/pic_5737.pn

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19398656 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19988480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 40108032 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 34865152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data)

cannot identify image file <_io.BufferedReader name='../data/validation/pic_61.png'>
61
cannot identify image file <_io.BufferedReader name='../data/validation/pic_4160.png'>
4160
cannot identify image file <_io.BufferedReader name='../data/validation/pic_6146.png'>
6146
cannot identify image file <_io.BufferedReader name='../data/validation/pic_972.png'>
972
cannot identify image file <_io.BufferedReader name='../data/validation/pic_1434.png'>
1434
cannot identify image file <_io.BufferedReader name='../data/validation/pic_1198.png'>
1198
tuple index out of range
5573
cannot identify image file <_io.BufferedReader name='../data/validation/pic_3324.png'>
3324
Only 2-D images (grayscale or color) are supported, when providing a callable `inverse_map`.
3330
cannot identify image file <_io.BufferedReader name='../data/validation/pic_4680.png'>
4680
tuple index out of range
2254
tuple index out of range
693
cannot identify image file <_io.BufferedReader name='../data/validation/pic_4140.pn

In [6]:
x_data = np.array([image[1] for image in val_images if image[1].shape[2] == 3])
x_data.shape

(5873, 299, 299, 3)

In [7]:
validation_image_ids = pd.DataFrame([image[0] for image in val_images if image[1].shape[2] == 3], columns=['val_image_id']) 

In [8]:
validation_image_labels = pd.concat([
    validation_image_ids.astype(int).set_index('val_image_id'),
    validation_labels.set_index('image_id')], join='inner', axis=1)

In [9]:
validation_image_labels_np = validation_image_labels.reset_index(drop=True).as_matrix()
validation_image_labels_np.shape

(5873, 1)

# Define model

In [10]:
from keras.models import Model
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions, InceptionV3

import numpy as np
import pandas as pd

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# load in preprocessed data

In [12]:
# x_data = validation_image_array

In [13]:
y_data = validation_image_labels_np

In [14]:
num_classes = 128

model = InceptionV3(weights='imagenet')

intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[311].output)

x = intermediate_layer_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

# train last cluster and dense layer
for layer in transfer_model.layers:
    layer.trainable = False

# can train more 
for i in range(311,313):
    transfer_model.layers[i].trainable = True

transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# using checkpoints and early stopping on validation sample to prevent overfitting
# best weight is saved to file_path
file_path="../checkpoints/weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
callbacks_list = [checkpoint, early] 

# Train model

In [15]:
epoch = 20

# onehot encoding
y_onehot = np.zeros((y_data.shape[0], num_classes))
for i in range(0,num_classes):
    (y_onehot[:,i:i+1])[y_data==i] = 1

transfer_model.fit(x_data, y_onehot, epochs=epoch, validation_split=0.1, batch_size=32, callbacks=callbacks_list,verbose=1)

# load best weights
transfer_model.load_weights(file_path)

Train on 5285 samples, validate on 588 samples
Epoch 1/20
5280/5285 [============================>.] - ETA: 0s - loss: 3.6203 - acc: 0.2275
Epoch 00001: val_loss improved from inf to 2.40386, saving model to ../checkpoints/weights_base.best.hdf5
5285/5285 [==============================] - 31s 6ms/step - loss: 3.6192 - acc: 0.2278 - val_loss: 2.4039 - val_acc: 0.4235
Epoch 2/20
5280/5285 [============================>.] - ETA: 0s - loss: 1.9940 - acc: 0.5186
Epoch 00002: val_loss improved from 2.40386 to 1.89976, saving model to ../checkpoints/weights_base.best.hdf5
5285/5285 [==============================] - 19s 4ms/step - loss: 1.9938 - acc: 0.5186 - val_loss: 1.8998 - val_acc: 0.4898
Epoch 3/20
5280/5285 [============================>.] - ETA: 0s - loss: 1.4353 - acc: 0.6244
Epoch 00003: val_loss improved from 1.89976 to 1.84647, saving model to ../checkpoints/weights_base.best.hdf5
5285/5285 [==============================] - 19s 4ms/step - loss: 1.4359 - acc: 0.6244 - val_loss: 1

# Apply to submission dataset

In [24]:
test_images_list = load_images('../data/test', (299, 299))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


cannot identify image file <_io.BufferedReader name='../data/test/pic_7670.png'>
7670
cannot identify image file <_io.BufferedReader name='../data/test/pic_8711.png'>
8711
tuple index out of range
2326
Could not load "" 
Reason: "image file is truncated (9 bytes not processed)"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
4559
tuple index out of range
2361
cannot identify image file <_io.BufferedReader name='../data/test/pic_7517.png'>
7517
cannot identify image file <_io.BufferedReader name='../data/test/pic_900.png'>
900
tuple index out of range
5992
tuple index out of range
3468
tuple index out of range
1679
Only 2-D images (grayscale or color) are supported, when providing a callable `inverse_map`.
10195
tuple index out of range
6619
cannot identify image file <_io.BufferedReader name='../data/test/pic_4413.png'>
4413
cannot identify image file <_io.BufferedReader name='../data/test/pic_5558.png'>
5558
tuple index out of 

cannot identify image file <_io.BufferedReader name='../data/test/pic_262.png'>
262
cannot identify image file <_io.BufferedReader name='../data/test/pic_7056.png'>
7056
tuple index out of range
6344
cannot identify image file <_io.BufferedReader name='../data/test/pic_12023.png'>
12023
cannot identify image file <_io.BufferedReader name='../data/test/pic_12280.png'>
12280
tuple index out of range
7493
cannot identify image file <_io.BufferedReader name='../data/test/pic_10708.png'>
10708
cannot identify image file <_io.BufferedReader name='../data/test/pic_3627.png'>
3627
tuple index out of range
5634
tuple index out of range
9769
cannot identify image file <_io.BufferedReader name='../data/test/pic_7080.png'>
7080
Could not load "" 
Reason: "image file is truncated (8 bytes not processed)"
Please see documentation at: http://pillow.readthedocs.org/en/latest/installation.html#external-libraries
2311
cannot identify image file <_io.BufferedReader name='../data/test/pic_8922.png'>
8922


In [ ]:
test_image_array = np.array(
    [image[1] 
     for image in test_images_list
     if image[1].shape[2] == 3])

In [30]:
test_predictions = [
    [x[0], transfer_model.predict(x[1].reshape([1, 299, 299, 3]))] 
    for x in test_images_list 
    if (len(x[1].shape) == 3 and x[1].shape[2] == 3)]

In [53]:
test_predictions_dict = dict([[int(x[0]), x[1]] for x in test_predictions])

In [45]:
list(test_predictions_dict.items())[0][1].argmax() + 1

26

In [55]:
def predict_or_1(x):
    return test_predictions_dict[x].argmax() if x in test_predictions_dict.keys() else 1

In [56]:
results = pd.DataFrame([[i, predict_or_1(i)] for i in range(1, 12801)], columns=['id', 'predicted'])

In [60]:
results.to_csv('first-submission.csv', index=False)

# Legacy code

In [3]:
model = InceptionV3(weights='imagenet')

intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[311].output)

x = intermediate_layer_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

# train last cluster and dense layer
for layer in transfer_model.layers:
    layer.trainable = False

# can train more 
for i in range(311,313):
    transfer_model.layers[i].trainable = True

transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

################### training last X layers ###################
epoch = 20
num_classes = 128
# onehot encoding
y_onehot = np.zeros((y_data.shape[0], num_classes))
for i in range(0,num_classes):
    (y_onehot[:,i:i+1])[y_data==i] = 1

# using checkpoints and early stopping on validation sample to prevent overfitting
# best weight is saved to file_path
file_path=".../weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
callbacks_list = [checkpoint, early] 

transfer_model.fit(x_data, y_onehot, epochs=epoch, validation_split=0.1, batch_size=32, callbacks=callbacks_list)

# load best weights
transfer_model.load_weights(file_path)

# need prediction

NameError: name 'model' is not defined

39952384/96112376 [===========>..................] - ETA: 17:04